In [1]:
# tf-idf for creative_id and advertiser_id
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

train_merged=pd.read_csv('train_merged.csv')
v = TfidfVectorizer(max_df=50,lowercase=False,tokenizer=lambda x:x)

creative_id=train_merged.groupby("user_id")["creative_id"].apply(list)
creative_id_matrix = v.fit_transform(creative_id)
display(creative_id_matrix)

sparse.save_npz('cr.npz', creative_id_matrix)

<900000x2414321 sparse matrix of type '<class 'numpy.float64'>'
	with 9965541 stored elements in Compressed Sparse Row format>

In [3]:
# predict gender
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse

user_info=pd.read_csv('user_info.csv')
creative_matrix=sparse.load_npz('cr.npz')
X_train=creative_matrix
y_train=user_info["gender"]
display(X_train,y_train)
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('gender.bin')

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 759 kB/s eta 0:00:01     |██████████████████              | 696 kB 759 kB/s eta 0:00:01


<900000x2414321 sparse matrix of type '<class 'numpy.float64'>'
	with 9965541 stored elements in Compressed Sparse Row format>

0         0
1         0
2         1
3         0
4         0
         ..
899995    0
899996    1
899997    1
899998    0
899999    1
Name: gender, Length: 900000, dtype: int64

In [ ]:
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd

train_data = lgb.Dataset('gender.bin')
lgb_params = {'num_leaves': 2**5-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start cv")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, shuffle=True,
    verbose_eval=1, show_stdv=True)

start cv
[1]	cv_agg's auc: 0.767025 + 0.000449946
[2]	cv_agg's auc: 0.826188 + 0.00121602
[3]	cv_agg's auc: 0.845422 + 0.000863808
[4]	cv_agg's auc: 0.851712 + 0.000605036
[5]	cv_agg's auc: 0.858422 + 0.00167923
[6]	cv_agg's auc: 0.866107 + 0.00223791
[7]	cv_agg's auc: 0.868168 + 0.00215166
[8]	cv_agg's auc: 0.869815 + 0.00173691
[9]	cv_agg's auc: 0.875981 + 0.00172246
[10]	cv_agg's auc: 0.879514 + 0.00162535
[11]	cv_agg's auc: 0.883363 + 0.00181844
[12]	cv_agg's auc: 0.885146 + 0.00141157
[13]	cv_agg's auc: 0.888497 + 0.0013869
[14]	cv_agg's auc: 0.890701 + 0.00106163
[15]	cv_agg's auc: 0.893231 + 0.000907936
[16]	cv_agg's auc: 0.894663 + 0.000914995
[17]	cv_agg's auc: 0.896867 + 0.00099255
[18]	cv_agg's auc: 0.898301 + 0.000900997
[19]	cv_agg's auc: 0.899725 + 0.000639843
[20]	cv_agg's auc: 0.901443 + 0.000951029
[21]	cv_agg's auc: 0.903209 + 0.000805491
[22]	cv_agg's auc: 0.904704 + 0.00076453
[23]	cv_agg's auc: 0.906559 + 0.000772696
[24]	cv_agg's auc: 0.907973 + 0.000716829
[25]	c

In [3]:
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('gender.bin')
lgb_params = {'num_leaves': 2**5-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start training")
model = lgb.train(lgb_params, train_data,num_boost_round=120)
model.save_model("gender_model.txt")
print("training completed!")

start training


KeyboardInterrupt: 

In [3]:
# predict gender
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

predict_merged=pd.read_csv('predict_merged.csv')
v = TfidfVectorizer(max_df=50,lowercase=False,tokenizer=lambda x:x)

creative_id=predict_merged.groupby("user_id")["creative_id"].apply(list)
creative_id_matrix = v.fit_transform(creative_id)
display(creative_id_matrix)
del predict_merged,creative_id,v

# sparse.save_npz('cr.npz', creative_id_matrix)
X_pred=creative_id_matrix
print("start predicting")
model=lgb.Booster(model_file='gender_model.txt')
y_pred=model.predict(X_pred)
y_pred=(y_pred>0.5)*1
submission=pd.DataFrame(y_pred,columns=["gender"])
submission["user_id"]=[i for i in range(user_info)]
display(submission)

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 759 kB/s eta 0:00:01     |██████████████████              | 696 kB 759 kB/s eta 0:00:01


<900000x2414321 sparse matrix of type '<class 'numpy.float64'>'
	with 9965541 stored elements in Compressed Sparse Row format>

0         0
1         0
2         1
3         0
4         0
         ..
899995    0
899996    1
899997    1
899998    0
899999    1
Name: gender, Length: 900000, dtype: int64